# Experimentos

In [1]:
import torch


In [2]:
def print_gpu_memory(prefix=""):
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024 ** 2)
        reserved = torch.cuda.memory_reserved() / (1024 ** 2)
        print(f"{prefix} Memory Allocated: {allocated:.2f} MB")
        print(f"{prefix} Memory Reserved: {reserved:.2f} MB")
    else:
        print("CUDA is not available.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() 

print_gpu_memory()

 Memory Allocated: 0.00 MB
 Memory Reserved: 0.00 MB


In [3]:
# imports

import os
import sys
sys.path.append(os.path.abspath('..'))

import src.models.unets as unets
import src.data.preprocess_data as data
import src.training.train_model as train

from torch.utils.data import DataLoader
import torch.optim as optim



In [4]:

tiles = ['032027', '032026'] 
num_subtiles = 6
classes_mode = 'type'

if classes_mode == 'type':
    num_classes = 5
elif classes_mode == 'density':
    num_classes = 4
elif classes_mode == 'binary':
    num_classes = 2
else:
    num_classes = 9




In [5]:
model_param_grid = {
    #model params:
    'model' : [f'UNet-256-{classes_mode}', #ok
               f'UNet-64-{classes_mode}', #ok
               f'UNetResNet34-224-{classes_mode}', #ok
               f'UNetEfficientNetB0-224-{classes_mode}', 
               f'UNetConvNext-224-{classes_mode}',
               ],
        
    #training params
        # loss
    'loss': ['BCE-dice'],#'BCE', 'dice', 'BCE-dice'],
    'weighted_loss': [False, True],
    'learning_rate': [0.0001],    
    'epochs' : [10],
    'patience' : [3],
    'batch_size' : [16]
       
}


In [6]:

train_files, val_files, test_files = data.train_val_test_stratify(tiles, 
                                                                  num_subtiles,
                                                                    train_size = 0.7, 
                                                                    val_size = 0.15, 
                                                                    stratify_by = classes_mode)



Loading:  <function yaml_filename at 0x70e079be2680>
File already saved, loading it.


In [ ]:
for model_params in train.iterate_parameter_grid(model_param_grid):

    #definitions
    model_name = model_params['model'] 
    patch_size = int(model_name.split('-')[1])
    print(model_params)
    model_class = model_name.split('-')[0]

    batch_size = model_params['batch_size'] 


    #load data
    train_dataset = data.SubtileDataset(train_files[:10], 
                    num_subtiles = num_subtiles, 
                    classes_mode = classes_mode, 
                    patch_size=(patch_size, patch_size), 
                    stride=patch_size//2, 
                    data_augmentation = True, 
                    return_imgidx = False,
                    return_nans=False,
                    debug=False)

    val_dataset = data.SubtileDataset(val_files[:10], 
                        num_subtiles = num_subtiles, 
                        classes_mode = classes_mode, 
                        patch_size=(patch_size, patch_size), 
                        stride=patch_size//2,
                        data_augmentation = False, 
                        return_imgidx = False,
                        return_nans=False,
                        debug=False)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    if model_name.startswith('UNet-'):
        model = unets.UNet(in_channels=12, out_channels=num_classes).to(device) 
    elif model_name.startswith('UNetResNet34-'):
        model = unets.UNetResNet34(in_channels=12, out_channels=num_classes).to(device) 
    elif model_name.startswith('UNetEfficientNetB0-'):
        model = unets.UNetEfficientNetB0(in_channels=12, out_channels=num_classes).to(device) 
    elif model_name.startswith('UNetConvNext-'):
        model = unets.UNetConvNext (in_channels=12, out_channels=num_classes).to(device) 
        
    optimizer = optim.Adam(model.parameters(), lr=model_params['learning_rate'], weight_decay=1e-5)
    train.train_model(model, 
                        train_loader, 
                        val_loader, 
                        epochs=model_params['epochs'], 
                        loss_mode = model_params['loss'],
                        optimizer = optimizer, 
                        device = device,
                        num_classes = num_classes, 
                        patience = model_params['patience'],
                        label_counts = None,
                        show_batches = 1, 
                        save_to = model_name+'.pth')

    break
1/0

    


{'model': 'UNet-256-type', 'loss': 'BCE-dice', 'weighted_loss': False, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3, 'batch_size': 16}
Doing data augmentation...
Augmented 453 images, of 1960
Model is already saved
Loading it e continue training...
LOSS mode: BCE-dice
Training for epoch 2 until epoch 10
Training Epoch 2/10
Running an epoch on the train mode.


 49%|████▉     | 157/321 [02:13<02:18,  1.18it/s]

In [ ]:

model = unets.UNet(12, out_channels=5).to(device)  # 12 channels, 5 classes


In [ ]:
for model_params in train.iterate_parameter_grid(model_param_grid):        
    print(model_params)
    #skip if model do not match the data
    model_name = model_params['model']     
    if f"-{patch_size}-" not in model_name:
        continue
    if f"{num_classes}classes" not in model_name:
        continue
            
    
    #label_count = None
    #if model_params['weighted_loss']:
    #    print('CLASSES COUNT')
    #   label_count = train_dataset.count_classes()
    
    # Training the model
    savename = f"models/{model_name}_{model_params['loss']}{'-w' if model_params['weighted_loss'] else ''}{'_normalized' if normalization=='normalized' else ''}_{'aug' if data_aug else 'noaug'}.pth"
    print(savename)
    if 1: #not os.path.exists(savename):
        if 0:
            for im, mask in train_loader:
                print(im.shape, type(im))
                print(mask.shape, type(mask))
                break
        
        if model_name.startswith('UNet-'):
            model = models.UNet(in_channels=12, out_channels=num_classes, dropout=0.2).to(device)  # Assuming single channel output for mask
        elif model_name.startswith('UNetResNet34-'):
            model = models.UNetResNet34(in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
        elif model_name.startswith('UNetEfficientNetB0-'):
            model = models.UNetEfficientNetB0(in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
        elif model_name.startswith('UNetConvNext-'):
            model = models.UNetConvNext (in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
        elif model_name.startswith('HRNetW18-'):
            model = models.HRNetSegmentation(in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
        else:
            print('Not recognized model. Skipping')
            continue

        optimizer = optim.Adam(model.parameters(), lr=model_params['learning_rate'], weight_decay=1e-5)
        train.train_model(model, 
                            train_loader, 
                            val_loader, 
                            epochs=model_params['epochs'], 
                            loss_mode = model_params['loss'],
                            optimizer = optimizer, 
                            device = device,
                            num_classes = num_classes, 
                            patience = model_params['patience'],
                            label_counts = train_stats,
                            show_batches = 1, 
                            save_to = savename)


In [ ]:
import src.view as view
import src.training_utils as train
import src.models as models
import src.process_data as data


from torch.utils.data import Dataset, DataLoader
import torch
import os
import random
import rasterio
import numpy as np
import torch.optim as optim
import json

batch_size = 16                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

device


[(np.float64(0.0), np.float64(0.5), np.uint64(2)), (np.float64(0.5), np.float64(3.0), np.uint64(2)), (np.float64(2.5), np.float64(0.0), np.uint64(2)), (np.float64(2.5), np.float64(2.5), np.uint64(4))]
[(np.float64(0.0), np.float64(0.5), np.uint64(2)), (np.float64(0.5), np.float64(3.0), np.uint64(2)), (np.float64(2.5), np.float64(0.0), np.uint64(2)), (np.float64(2.5), np.float64(2.5), np.uint64(4))]


device(type='cuda')

In [ ]:
train_files, val_files, test_files = data.train_val_test_stratify(images_folder='data/processed/36_subtile_divisions', 
                            train_size = 24, 
                            val_size = 6, 
                            test_size = 6,
                            labels_file="masks/raster_aurb_032027.tif",
                            save_json = 'subtiles_filenames.json',
                            subtile_size = 10560/6,
                            check_stratification = True)


['data/processed/36_subtile_divisions/S2-16D_V2_032027_x=7040_y=1760.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=5280_y=7040.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=5280_y=3520.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=8800_y=5280.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=1760_y=3520.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=7040_y=8800.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=3520_y=0.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=8800_y=0.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=1760_y=0.tif']
['data/processed/36_subtile_divisions/S2-16D_V2_032027_x=0_y=1760.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=5280_y=8800.tif']
['data/processed/36_subtile_divisions/S2-16D_V2_032027_x=5280_y=5280.tif', 'data/processed/36_subtile_divisions/S2-16D_V2_032027_x=0_y=0.tif']
Loaded data from JSON file.
train set ([label][number

In [ ]:

data_param_grid = {
    'patch_size' : [256, 64, 224, 512],
    #'patch_size' : [224, 512],
    'num_classes' : [2,5],
    'normalization' : ['unnormalized', 'normalized'],
    'data_aug' : [False],#True
}

model_param_grid = {
    #model params:
    'model' : ['UNet-256-2classes', #ok
               'UNet-64-2classes', #ok
               'UNetResNet34-224-2classes', #ok
               'UNetEfficientNetB0-224-2classes', 
               'UNetConvNext-224-2classes',
               #'HRNetW18-512-2classes',
               'UNet-256-5classes', #ok
               'UNet-64-5classes', #ok
               'UNetResNet34-224-5classes', #ok
               'UNetEfficientNetB0-224-5classes', 
               'UNetConvNext-224-5classes',
               #'HRNetW18-512-5classes'
               ],
        
    #training params
        # loss
    'loss': ['BCE-dice'],#'BCE', 'dice', 'BCE-dice'],
    'weighted_loss': [False, True],
    'learning_rate': [0.0001],    
    'epochs' : [10],
    'patience' : [3],
       
}

#1/0
# Grid parameter loop for training
# Data loop
for data_params in train.iterate_parameter_grid(data_param_grid):
    print(data_params)

    normalization = data_params['normalization']

    if normalization == 'normalized': 
        mean, std = data.calculate_mean_std(train_files)
    else:
        mean, std = None, None
    
    data_aug = False
    if data_params['num_classes']>2:
        data_aug = data_params['data_aug']

    patch_size = data_params['patch_size']
    stride = int(patch_size*(2/3))
    num_classes = data_params['num_classes']

    print(patch_size)
    train_dataset = data.ImageDataset(files = train_files, 
                            mask_filename = "masks/raster_aurb_032027.tif", 
                            num_classes = num_classes,
                            subtile_size = 10560/6,
                            mean = mean, 
                            std = std,
                            patch_size = (patch_size, patch_size),
                            stride = stride,
                            augment = data_aug)
    if data_aug:
        train_dataset.check_augmentation()

    val_dataset = data.ImageDataset(files = val_files, 
                        mask_filename = "masks/raster_aurb_032027.tif", 
                        num_classes = num_classes,
                        mean = mean, 
                        std = std,
                        patch_size = (patch_size, patch_size),
                        stride = stride,
                        augment = False)


    train_loader = DataLoader(train_dataset, 
                            batch_size=batch_size, 
                            shuffle=True)
    val_loader = DataLoader(val_dataset, 
                            batch_size=batch_size, 
                            shuffle=True)
    train_stats = None
    val_stats = None
    if data_aug:
        train_stats = data.count_labels(train_loader)
        print('TRAIN STATS:',train_stats)
        val_stats = data.count_labels(val_loader)
        print('VAL STATS:',val_stats)
    
    if 0:
        for im_batch,label_batch in train_loader:
            if all(label_batch.sum()>0 for i in range(label_batch.shape[0])):
                for i in range(label_batch.shape[0]): 
                    if label_batch[i].sum()>0:
                        view.plot_batch(im_batch, label_batch)
                        break

    # Models loop
    for model_params in train.iterate_parameter_grid(model_param_grid):        
        print(model_params)
        #skip if model do not match the data
        model_name = model_params['model']     
        if f"-{patch_size}-" not in model_name:
            continue
        if f"{num_classes}classes" not in model_name:
            continue
                
        
        #label_count = None
        #if model_params['weighted_loss']:
        #    print('CLASSES COUNT')
        #   label_count = train_dataset.count_classes()
        
        # Training the model
        savename = f"models/{model_name}_{model_params['loss']}{'-w' if model_params['weighted_loss'] else ''}{'_normalized' if normalization=='normalized' else ''}_{'aug' if data_aug else 'noaug'}.pth"
        print(savename)
        if 1: #not os.path.exists(savename):
            if 0:
                for im, mask in train_loader:
                    print(im.shape, type(im))
                    print(mask.shape, type(mask))
                    break
            
            if model_name.startswith('UNet-'):
                model = models.UNet(in_channels=12, out_channels=num_classes, dropout=0.2).to(device)  # Assuming single channel output for mask
            elif model_name.startswith('UNetResNet34-'):
                model = models.UNetResNet34(in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
            elif model_name.startswith('UNetEfficientNetB0-'):
                model = models.UNetEfficientNetB0(in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
            elif model_name.startswith('UNetConvNext-'):
                model = models.UNetConvNext (in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
            elif model_name.startswith('HRNetW18-'):
                model = models.HRNetSegmentation(in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
            else:
                print('Not recognized model. Skipping')
                continue

            optimizer = optim.Adam(model.parameters(), lr=model_params['learning_rate'], weight_decay=1e-5)
            train.train_model(model, 
                                train_loader, 
                                val_loader, 
                                epochs=model_params['epochs'], 
                                loss_mode = model_params['loss'],
                                optimizer = optimizer, 
                                device = device,
                                num_classes = num_classes, 
                                patience = model_params['patience'],
                                label_counts = train_stats,
                                show_batches = 1, 
                                save_to = savename)


{'patch_size': 256, 'num_classes': 2, 'normalization': 'unnormalized', 'data_aug': False}
256
{'model': 'UNet-256-2classes', 'loss': 'BCE-dice', 'weighted_loss': False, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3}
models/UNet-256-2classes_BCE-dice_noaug.pth
Model is already saved
{'model': 'UNet-256-2classes', 'loss': 'BCE-dice', 'weighted_loss': True, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3}
models/UNet-256-2classes_BCE-dice-w_noaug.pth
Model is already saved
{'model': 'UNet-64-2classes', 'loss': 'BCE-dice', 'weighted_loss': False, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3}
{'model': 'UNet-64-2classes', 'loss': 'BCE-dice', 'weighted_loss': True, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3}
{'model': 'UNetResNet34-224-2classes', 'loss': 'BCE-dice', 'weighted_loss': False, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3}
{'model': 'UNetResNet34-224-2classes', 'loss': 'BCE-dice', 'weighted_loss': True, 'learning_rate': 0.0001, 'epochs': 10, 

1it [00:00,  1.79it/s]

(12, 1760, 1760) 0.1464993 0.1036741 37151964 37171200 (12, 1760, 1760)


2it [00:01,  1.16it/s]

(12, 1760, 1760) 0.15382166 0.100056104 37049244 37171200 (12, 1760, 1760)


...

100%|██████████| 216/216 [01:34<00:00,  2.29it/s]


Train Loss: 0.4232741152798688
Train Accuracy: 0.989617109298706
              precision    recall  f1-score    support
Class 0        0.993726  0.995025  0.994375  159818128
Class 1        0.939970  0.925378  0.932617   13454138
weighted avg   0.989552  0.989617  0.989580  173272272
macro avg      0.966848  0.960201  0.963496  173272272
micro avg      0.989617  0.989617  0.989617  173272272
Running an epoch on the validation mode.


100%|██████████| 54/54 [00:11<00:00,  4.59it/s]


Train Loss: 0.6255403327997084
Train Accuracy: 0.9853901863098145
              precision    recall  f1-score   support
Class 0        0.989434  0.995202  0.992310  41034232
Class 1        0.904100  0.809748  0.854327   2292241
weighted avg   0.984919  0.985390  0.985009  43326472
macro avg      0.946767  0.902475  0.923318  43326472
micro avg      0.985390  0.985390  0.985390  43326472
{'model': 'UNet-256-5classes', 'loss': 'BCE-dice', 'weighted_loss': False, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3}
{'model': 'UNet-256-5classes', 'loss': 'BCE-dice', 'weighted_loss': True, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3}
{'model': 'UNet-64-5classes', 'loss': 'BCE-dice', 'weighted_loss': False, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3}
{'model': 'UNet-64-5classes', 'loss': 'BCE-dice', 'weighted_loss': True, 'learning_rate': 0.0001, 'epochs': 10, 'patience': 3}
{'model': 'UNetResNet34-224-5classes', 'loss': 'BCE-dice', 'weighted_loss': False, 'learning_rate'

RuntimeError: Error(s) in loading state_dict for UNetResNet34:
	size mismatch for upconv1.weight: copying a param with shape torch.Size([64, 64, 2, 2]) from checkpoint, the shape in current model is torch.Size([64, 64, 1, 1]).

In [ ]:
# Grid parameter loop for training
# Data loop
for data_params in train.iterate_parameter_grid(data_param_grid):
    print(data_params)

    normalization = data_params['normalization']

    if normalization == 'normalized': 
        mean, std = data.calculate_mean_std(train_files)
    else:
        mean, std = None, None
    
    data_aug = False
    if data_params['num_classes']>2:
        data_aug = data_params['data_aug']

    patch_size = data_params['patch_size']
    num_classes = data_params['num_classes']
    test_dataset = data.ImageDataset(files = test_files[:1], 
                mask_filename = "masks/raster_aurb_032027.tif", 
                num_classes = num_classes,
                mean = mean, 
                std = std,
                patch_size = (patch_size, patch_size),
                stride = patch_size//2,
                augment = False,
                return_imgidx = True)

    test_loader = DataLoader(test_dataset, 
                                batch_size=batch_size, 
                                shuffle=True)

        #test_stats = data.count_labels(test_loader)
        #print(test_stats)
    for model_params in train.iterate_parameter_grid(model_param_grid):        
        
        #skip if model do not match the data
        model_name = model_params['model']     
        if f"-{patch_size}-" not in model_name:
            continue
        if f"{num_classes}classes" not in model_name:
            continue
                
        savename = f"models/{model_name}_{model_params['loss']}{'-w' if model_params['weighted_loss'] else ''}{'_normalized' if normalization=='normalized' else ''}_{'aug' if data_aug else 'noaug'}.pth"
        print(savename)
    
        
        if model_name.startswith('UNet-'):
            model = models.UNet(in_channels=12, out_channels=num_classes, dropout=0.2).to(device)  # Assuming single channel output for mask
        elif model_name.startswith('UNetResNet34-'):
            model = models.UNetResNet34(in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
        elif model_name.startswith('UNetEfficientNetB0-'):
            model = models.UNetEfficientNetB0(in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
        elif model_name.startswith('UNetConvNext-'):
            model = models.UNetConvNext (in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
        elif model_name.startswith('HRNetW18-'):
            model = models.HRNetSegmentation(in_channels=12, out_channels=num_classes).to(device)  # Assuming single channel output for mask
        else:
            print('Not recognized model. Skipping')
            continue

        results, gt, originals = train.test_model(model, savename, test_loader, device, num_classes, subtile_size = int(10560/6))
        
        break
    break
    #plot_multiple_satellite_images


In [ ]:
ims = []
for f in originals:
    with rasterio.open(f) as im:        
        #window = rasterio.windows.Window(x+subtile_x ,y+subtile_y , patch_size, patch_size) #TODO mudar esse hardcoded
        im = im.read()#window = window)
        im = im[1:4,:,:]
        im = im.transpose((1, 2, 0))
        im = np.clip(im, 0, 10000)
        im = im-np.min(im)
        im = im/np.max(im)
        ims.append(im)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

limx = (1000,1700)
limy = (1000,1700)
for im, mask in zip(ims, results):
    plt.figure(figsize=(10,10))
    plt.imshow(im[limx[0]:limx[1],limy[0]:limy[1],:])
    for c in range(5):
        plt.figure(figsize=(10,10))
        plt.imshow(mask[c,limx[0]:limx[1],limy[0]:limy[1]], cmap='gray')
    print(np.min(mask), np.max(mask), mask.shape)
    #plt.figure(figsize=(10,10))
    #plt.imshow(mask>0.5)




In [ ]:



import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

def plot_satellite_with_mask(ax, satellite_image, mask, title="Satellite Image with Mask Overlay",
                             mask_color=(1, 0, 0), mask_alpha=0.5, border_color='red',
                             border_width=2):

    if satellite_image.shape[-1]!=3:
        satellite_image = satellite_image[1:4,:,:]
        satellite_image = satellite_image.transpose((1, 2, 0))
    # Ensure mask is binary
    mask = (mask > 0).astype(int)

    # Create a custom colormap for the mask
    colors = [(0, 0, 0, 0), mask_color + (mask_alpha,)]
    cmap = LinearSegmentedColormap.from_list("custom", colors, N=2)

    # Plot the satellite image
    ax.imshow(satellite_image)
    print(np.min(satellite_image))
    print(np.max(satellite_image))
    # Plot the mask overlay
    #mask_plot = ax.imshow(mask, cmap=cmap)

    # Plot the mask border
    #ax.contour(mask, colors=border_color, linewidths=border_width, levels=[0.5])

    # Set title and remove axis ticks
    ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])

    return ax
    return mask_plot

def plot_multiple_satellite_images(satellite_images, masks, titles=None, figsize=(20, 20),
                                   mask_color=(1, 0, 0), mask_alpha=0.5, border_color='red',
                                   border_width=2):
    """
    Plot multiple satellite images with their corresponding masks in subplots.

    Parameters:
    satellite_images (list): List of satellite images (each a 3D numpy array)
    masks (list): List of corresponding masks (each a 2D numpy array)
    titles (list): List of titles for each subplot (optional)
    figsize (tuple): Figure size in inches (width, height)
    mask_color (tuple): RGB color for the mask (values between 0 and 1)
    mask_alpha (float): Alpha value for mask transparency (between 0 and 1)
    border_color (str or tuple): Color of the mask border
    border_width (float): Width of the mask border

    Returns:
    matplotlib.figure.Figure: The created figure
    """
    n = len(satellite_images)
    rows = int(np.ceil(np.sqrt(n)))
    cols = int(np.ceil(n / rows))

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    if n == 1:
        axes = np.array([axes])
    axes = axes.flatten()

    for i, (image, mask) in enumerate(zip(satellite_images, masks)):
        if i < n:
            title = titles[i] if titles and i < len(titles) else f"Image {i+1}"
            mask_plot = plot_satellite_with_mask(axes[i], image, mask, title, 
                                                 mask_color, mask_alpha, 
                                                 border_color, border_width)
        else:
            axes[i].axis('off')

    # Add a colorbar to the last subplot
    cbar = fig.colorbar(mask_plot, ax=axes[-1], ticks=[0, 1])
    cbar.set_label('Mask Value')

    plt.tight_layout()
    return fig




In [ ]:
fig = view.plot_multiple_satellite_images(ims, results)

plt.show()

In [ ]:
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

# Define the size of the grid
grid_size = (10, 10)
grid = np.full(grid_size, np.nan)  # Initialize a grid with NaN values (for missing points)

# Populate the grid with some known values
known_points = [
    (0, 0, 1),
    (0, 9, 1),
    (9, 0, 0),
    (9, 9, 0)
]

for x, y, value in known_points:
    grid[y, x] = value

# Extract the coordinates and values of known points
y_coords, x_coords = np.where(~np.isnan(grid))  # Only coordinates with values
values = grid[y_coords, x_coords]

# Create a full grid for interpolation
x_grid, y_grid = np.meshgrid(np.arange(grid_size[1]), np.arange(grid_size[0]))

# Perform interpolation to fill in missing values
filled_grid = griddata(
    points=(x_coords, y_coords), values=values, 
    xi=(x_grid, y_grid), method='linear'
)

# Plot original grid with NaNs and the filled grid
plt.subplot(1, 2, 1)
plt.title("Original Grid (with NaNs)")
plt.imshow(grid, cmap='viridis', origin='lower')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title("Filled Grid (Interpolated)")
plt.imshow(filled_grid, cmap='viridis', origin='lower')
plt.colorbar()

plt.show()


In [ ]:
import os
import sys
sys.path.insert(0, 'src')

import test_utils as test

patches = []
counter = 0
for i in range(0, 256, 128):
    for j in range(0, 256, 128):
        patch = {'patch_idx' : counter,
                     'x':x,
                     'y':y,
                     'w':356,
                     'h':256}
        patches.append(patch)
        counter +=1

regions = test.get_regions(patches)

#grid = test.make_grid(regions, (256,256))

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Load the satellite image and mask
# Replace these with your actual image loading code
satellite_image = np.random.rand(100, 100, 3)  # Placeholder for satellite image
mask = np.random.randint(0, 2, (100, 100))  # Placeholder for mask

# Create a custom colormap for the mask
colors = [(0, 0, 0, 0), (1, 0, 0, 0.5)]  # Black (transparent) for 0, semi-transparent red for 1
cmap = LinearSegmentedColormap.from_list("custom", colors, N=2)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot the satellite image
ax.imshow(satellite_image)

# Plot the mask overlay
mask_plot = ax.imshow(mask, cmap=cmap)

# Add a colorbar for the mask
cbar = plt.colorbar(mask_plot, ax=ax, ticks=[0, 1])
cbar.set_label('Mask Value')

# Plot the mask border
ax.contour(mask, colors='red', linewidths=2, levels=[0.5])

# Set title and remove axis ticks
ax.set_title('Satellite Image with Mask Overlay')
ax.set_xticks([])
ax.set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
y_true, y_pred = [1,0,0],[1,1,0]
acc_class_report(y_true, y_pred)

In [ ]:
report_df

In [ ]:
patches com 10% dos dados?
data aug so nas minoritarias
mostrar os erros
olhar o recall
salvar as mascaras para abrir no qgis.
aplicar só no setor censitario?



qual a saida do modelo? probabilidade de cada classes?
qual data augmentation vc fez?
qual as imagens vc ta usando? sentinel? resolucao?
como é a balanced accuracy?

0
loteamento vazio
outros equipamentos
vazio interubano
area urbanizada
